# Elastic Net Regression - BSQF Assessment

This project is a sample, done by Bocconi Students Quantitative Finance, for the Fall 2025 Recruitment Process.
In this project we are going to:
- Theoretical introduction to the topic
- Download weekly data for 20-30 stocks
- Build features like momentum, rolling volatility and size
- Train Elastic Net with cross-validation to choose alpha and l1_ratio
- Compare results with other regressions' (OLS, Lasso, Ridge) ability to predict next week return

### Part 1: Introduction

One of the main risk, when running a linear regression is "Overfitting" which is the risk that the model adapts too much to the data and becomes unable to generalize.
Lasso and Ridge regressions tackle this problem by exploiting the bias-variance trade-off, introducing bias but decreasing varianca, that resulting in a reduction in MSE.  
This method mainly works with big databases where explaining variables could be collinear and not always useful, thus attributing high values to this variables could result in a misinterpretation of their impact on the dependent variable, thus these models aims to mitigate coefficient values and, eventually, eliminate them.  
A further sophistication of the Lasso and Ridge model is the Elastic Net regresion which combine this two models and now we will see how it works precisely.

### Part 2: Gather market data

In [27]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [32]:
# We start by downloading real stocks' data
tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA",
    "JPM", "NVDA", "XOM", "KO", "BA",
    "PFE", "CVX", "DIS", "NFLX", "INTC",
    "NKE", "MCD", "COST", "AMD", "GS"
]
tickers.sort()
start = "2020-01-01" 
end = "2025-10-06"
data = yf.download(tickers, start=start, end=end)

shares_table=pd.DataFrame(index=data.index)
for ticker in tickers:
    shares_table[f"{ticker}"] = yf.Ticker(ticker).info.get("sharesOutstanding", None)

C:\Users\alema\AppData\Local\Temp\ipykernel_13300\1628578379.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start, end=end)
[*********************100%***********************]  20 of 20 completed


In [33]:
# Let's see how our data looks like
data.head()

Price           Close                                                \
Ticker           AAPL        AMD       AMZN          BA        COST   
Date                                                                  
2020-01-02  72.538506  49.099998  94.900497  331.348572  266.874512   
2020-01-03  71.833282  48.599998  93.748497  330.791901  267.094269   
2020-01-06  72.405693  48.389999  95.143997  331.766083  267.167542   
2020-01-07  72.065125  48.250000  95.343002  335.285156  266.746368   
2020-01-08  73.224411  47.830002  94.598503  329.410095  269.804382   

Price                                                                ...  \
Ticker            CVX         DIS      GOOGL          GS       INTC  ...   
Date                                                                 ...   
2020-01-02  93.955627  145.769882  67.965225  204.189880  53.666466  ...   
2020-01-03  93.630661  144.097794  67.609680  201.802216  53.013714  ...   
2020-01-06  93.313423  143.261703  69.411758  203.867432  52.863766  ...   
2020-01-07  92.121872  143.310898  69.277687  205.209473  51.981667  ...   
2020-01-08  91.069550  143.015808  69.770782  207.187561  52.016956  ...   

Price         Volume                                                 \
Ticker           JPM        KO      MCD      MSFT     NFLX      NKE   
Date                                                                  
2020-01-02  10803700  11867700  3554200  22622100  4485800  5644100   
2020-01-03  10386800  11354500  2767600  21116200  3806900  4541800   
2020-01-06  10259000  14698300  4660400  20813700  5663100  4612400   
2020-01-07  10531300   9973900  4047400  21634100  4703200  6719900   
2020-01-08   9695300  10676000  5284200  27746500  7104500  4942200   

Price                                                 
Ticker           NVDA       PFE       TSLA       XOM  
Date                                                  
2020-01-02  237536000  16514072  142981500  12456400  
2020-01-03  205384000  14922848  266677500  17386900  
2020-01-06  262636000  15771951  151995000  20081900  
2020-01-07  314856000  20108107  268231500  17387700  
2020-01-08  277108000  16403507  467164500  15137700  

[5 rows x 100 columns]

#### Value selection

By using yt download method we are supplied with a 5 different values: Open, Close, High, Low and Volume. As we want to predict price, not peaks or quantity, we avoig using High, Low and Volume.  
Chosing between Open and Close, we prefer Close as its the settlement price for a certain security, specifically we're going to try to predict Close Adj. Price as it isn't affected by dividends and stock splits. 

In [59]:
target = np.log(data["Close"]/data["Close"].shift(1)).iloc[-1]

In [58]:
# We take into account Close_Adj. (with the new yf, there's no distinction between Close and Close_Adj.)
close = data["Close"]

cap_table=pd.DataFrame(index=data.index)
for i in range(close.shape[1]):
    cap_table[tickers[i]]=close.iloc[:, i]*shares_table.iloc[:, i]

for i in range(close.shape[1]):
    work_data = close.iloc[:, i]
    log_rets = np.log(work_data/work_data.shift(1))
    close[f"{data.columns[i][1]}_log_return"] = log_rets
    momentum = work_data.shift(21)/work_data.shift(252)-1
    close[f"{data.columns[i][1]}_momentum"] = momentum
    r_volatility = work_data.rolling(window=84).std()   
    close[f"{data.columns[i][1]}_roll_vol"] = r_volatility
    close[f"{data.columns[i][1]}_cap"] = cap_table.iloc[:, i]
close = close.dropna()
close = close.drop("2025-10-03")

C:\Users\alema\AppData\Local\Temp\ipykernel_13300\1366591218.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close[f"{data.columns[i][1]}_log_return"] = log_rets
C:\Users\alema\AppData\Local\Temp\ipykernel_13300\1366591218.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close[f"{data.columns[i][1]}_momentum"] = momentum
C:\Users\alema\AppData\Local\Temp\ipykernel_13300\1366591218.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

### Please Note:

Now we have our target variable, but we are going to use the entire database nevertheless, as more data can supply more information for target prediction and, additionally, our model it's made for handling big size databases (surely even bigger than the one we're going to use).